# PIPELINE RESUME DEEP LEARNING

**Bonjour** et **bienvenue** dans ce Jupyter de pipeline résumé via des méthodes de Deep Learning. Veillez bien à avoir le fichier __fats.py__ dans votre dossier pour l'importer correctement. Décommenter la 1ère cellule si besoin pour installer les packages requis.

Vérifiez de bien avoir le fichier __MLSUM_fr_test.pickle__ (ou un autre fichier contenant des paragraphes à résumer, attention à la dimension) ainsi que le tokenizer (ou un tokenizer, suivant le texte que vous utilisez).

In [ ]:
# A faire tourner que si vous n'avez pas déjà installé les packages nécessaires
!pip install gensim==3.8.3
!pip install unidecode
!pip install torch
!pip install sentencepiece
!pip install transformers
!pip install bs4
!pip install networkx

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pickle
import os
os.chdir('C:\\Users\\theo.roudil-valentin\\Documents\\Codes')
os.chdir("2021-NLP_AE\\Theo") #Indiquez votre chemin vers votre dossier correspondant

import fats

In [5]:
texte=pickle.load(open('Paragraphes_exemple.pickle','rb')) # Veillez à bien avoir le fichier test en question pour pouvoir reproduire l'exemple

tok='MLSUM_tokenizer.model' # choix du tokenizer

In [6]:
r=fats.make_DL_resume(texte, # une liste de phrases ou une liste de listes de phrases.
                      cpu=2, # nombre de cpu à utiliser, je conseille peu de cpu pour garder de la mémoire.
                      choose_model='SMHA', # le nom du modèle à aller chercher. si vous ne les connaissez pas, 
                      #ne mettez rien et l'erreur affichera les noms des modèles.
                      k=3, # nombre de phrases à retenir
                      camem=None, # le modèle camembert que vous utilisez, par défaut le modèle from_pretrained('camembert_base') sera utilisé
                      vs=12000, # taille du vocabulaire pour l'encoding, paramètre pour le tokenizer. Fixé à 12000 par convenance.
                      sp=1, # split, fixé à 1 car cette fonction n'est pas pour l'entraînement.
                      tok=tok, # le chemin vers le tokenizer, par défaut 'MLSUM_tokenizer.model', 
                      #faîtes donc bien attention à changer le chemin os vers le bon dossier au début du Notebook
                      tr=False, # dummy pour l'entraînement, fixé à False donc.
                      get_score_only=False) # dummy pour ne récupérer que les index des phrases et non les phrases elles-mêmes.

ValueError: Attention, ce que vous avez entré pour la variable choose_model ne semble pas convenir. Vous ne possédez probablement pas le modèle dans votre dossier ou l'avez mal rebaptisé. Les noms corrects sont les suivants :
SMHA_Linear_classifier.pt,
Simple_Classifier.pt,
Net.pt,
Multi_Linear_Classifier.pt

In [4]:
r # affiche le ou les résumés

[[' CARL RECINE / REUTERS Liverpool au sommet de l’Europe… et de l’ennui',
  ' Les Reds, pour ne pas se découvrir, ont allongé le jeu et outre l’intenable Salah (21e, 22e, 39e), seuls les latéraux Trent Alexander-Arnold, d’une frappe trop croisée (17e), et Andy Robertson, d’un tir claqué par Lloris (38e), ont instillé quelques frissons dans la torpeur ambiante',
  ' En 2018, les bourdes du gardien Loris Karius avaient plombé Liverpool'],
 [' C’est au contraire comme un vagabond ami de la Terre, arpenteur inventif, ouvert aux fécondités du hasard',
  ' Michel Serres aura sans doute rendu à la philosophie française son sens de la rencontre, de l’imprévu, du jeu',
  ' Une affaire de style plutôt, et de tempérament'],
 [' Or Michel Serres fut un grand voyageur – ce qui lui permit d’être, aussi, un prodigieux conteur d’histoires',
  ' Michel Serres est décédé samedi 1er juin, à l’âge de 88 ans',
  ' A l’intérieur de celui-ci, le plus petit opuscule, le moindre sous-système reproduit la stru

## On prend en exemple un ensemble d'articles

In [ ]:
from transformers import CamembertModel
camem2=CamembertModel.from_pretrained("camembert-base")

from transformers import CamembertTokenizer
from unidecode import unidecode
from bs4 import BeautifulSoup
import gensim
from functools import partial
import sentencepiece as spm 
import psutil
from tqdm import tqdm
import time
import functools
import operator
from joblib import Parallel,delayed
from joblib.parallel import cpu_count
import pandas as pd
import numpy as np

In [3]:
text=Parallel(n_jobs=1)(delayed(fats.make_text)(t) for t in texte)
text[0][1]


#dico=fats.Make_Extractive(5).make_encoding(text,voc_size=12000,split=1,tokenizer=tok,training=False)

[13, 14, 15, 16]

In [49]:
a=[i for i in range(len(texte[0]))]
for i in t1ouf:
    a.remove(i)

In [ ]:
b=[i for i in range(len(a))]
b

In [29]:
SMHAL=fats.SMHA_Linear_classifier(torch.Size([512,768]),8,768)
SMHAL.load_state_dict(torch.load('SMHA_Linear_classifier.pt',map_location=torch.device('cpu')))

<All keys matched successfully>

In [30]:
SMHAL.eval()
y=SMHAL(camem2(torch.Tensor(dico['input']).long(),torch.Tensor(dico['mask']).long()).last_hidden_state)

In [31]:
t=torch.mul(dico['mask_cls'],y).topk(3)
values=t[0]
indice=t[1]
vec=[(dico['mask_cls'][i]==torch.tensor(1)).nonzero() for i in range(len(dico['mask_cls']))]

In [32]:
Ve=[]
V=[]
I=[]
ind=0
for i in range(len(dico['trace'])):
    Ve.append(vec[ind:ind+dico['trace'][i]])
    V.append(values[ind:ind+dico['trace'][i]])
    I.append(indice[ind:ind+dico['trace'][i]])
    ind+=dico['trace'][i]
V=[v.reshape(-1) for v in V]
I=[v.reshape(-1) for v in I]
I2=[torch.cat([I[k][i*3:(i+1)*3]+512*i for i in range(int(len(I[k])/3))]) for k in range(len(I))]
Ve2=[torch.cat([Ve[k][i]+512*i for i in range(len(Ve[k]))]).squeeze(1) for k in range(len(Ve))]
index=[torch.div(V[i],V[i].sum()).topk(k=3)[1] for i in range(len(V))]
vrai_index=[I2[i][index[i]] for i in range(len(texte))]
rindex=[torch.cat([(Ve2[k]==vrai_index[k][i]).nonzero().squeeze(1) for i in range(3)]) for k in range(len(Ve2))]

In [33]:
rindex

[tensor([ 1, 18, 15]),
 tensor([19, 20,  4]),
 tensor([ 2, 88, 17]),
 tensor([ 2,  3, 23]),
 tensor([ 3, 10, 16]),
 tensor([ 9,  4, 10]),
 tensor([ 1, 11, 10]),
 tensor([ 1,  8, 10]),
 tensor([12, 18, 11]),
 tensor([52, 18,  2])]

In [37]:
for k in range(len(text)):
    for i in rindex[k]:
        try:
            text[k][i]
        except:
            print(k,i)
            continue

In [196]:
resu=[[text[i][k] for k in rindex[i]] for i in range(len(text))]
resu

[['carl recine reuters liverpool sommet europe ennui',
  'okepamzchv rmcsport rmc sport mais cinquieme but egyptien dans cette qui aurait enflammer rencontre contraire assomme tout monde',
  'tout avait pourtant commence par coup froid des premiere action debordement sadio mane centre detourne epaule puis bras par sissoko penalty siffle apres seulement jeu plus rapide dans histoire des finales'],
 ['est contraire comme vagabond ami terre arpenteur inventif ouvert aux fecondites hasard',
  'michel serres aura sans doute rendu philosophie francaise son sens rencontre imprevu jeu',
  'une affaire style plutot temperament'],
 ['michel serres est decede samedi juin age ans', 'demission retour lycee'],
 ['domine physiquement febrile premiere periode lou fini par etouffer montpellier barrages top samedi juin dans fournaise gerland bout suspense',
  'les lyonnais qualifient pour les demi finales pour seconde annee consecutive'],
 ['plusieurs elus locaux berlinois des membres bundestag ainsi qu

In [ ]:
[torch.cat([(Ve2[k]==vrai_index[k][i]).nonzero().squeeze(1) for i in range(3)]) for k in range(len(Ve2))]

In [ ]:
(Ve2==vrai_index).nonzero()

In [ ]:
vec=[(dico['mask_cls'][i]==torch.tensor(1)).nonzero() for i in range(len(dico['mask_cls']))]
rindex=[torch.transpose((vec[i]==t[1][i]).nonzero(),0,1)[0] for i in range(len(t[1]))]
rindex

## Pour différents paragraphes

In [ ]:
fichiers=[i for i in os.listdir() if (i[-6:]=='pickle') and (i[:5]=='MLSUM')]
fichiers_val=[i for i in fichiers if 'test' in i]
test=pickle.load(open('MLSUM_fr_test.pickle','rb'))
texte=[test.text[:10].values.tolist()[i].split('.') for i in range(10)] #Liste de liste de phrases

In [4]:
def make_DL_resume(texte,cpu,choose_model,k=3,camem=None,vs=12000,sp=1,tok='MLSUM_tokenizer.model',tr=False,get_score_only=False):
    try :
        assert choose_model in ['SMHA','Simple','Net','Multi']
    except:
        raise ValueError("Attention, le nom de votre modèle n'est pas correcte, il doit faire partie des noms suivants :\nSMHA (pour Self Multi Head Attention),\nSimple (pour Simple Linear Classifier),\nNet (pour Convolutional Network),\nMulti (pour Multi Linear)")
    
    try:
        assert np.sum([1 if choose_model in i else 0 for i in os.listdir()])>0
    except:
        raise ValueError("Attention, ce que vous avez entré pour la variable choose_model ne semble pas convenir. Vous ne possédez probablement pas le modèle dans votre dossier ou l'avez mal rebaptisé. Les noms corrects sont les suivants :\nSMHA_Linear_classifier.pt,\nSimple_Classifier.pt,\nNet.pt,\nMulti_Linear_Classifier.pt")
        
    if camem==None:
        from transformers import CamembertModel
        camem=CamembertModel.from_pretrained("camembert-base")
        
    Text=Parallel(n_jobs=cpu)(delayed(fats.make_text)(t) for t in texte)
    text=[i[0] for i in Text]
    empty=[i[1] for i in Text]
    dico=fats.Make_Extractive(cpu).make_encoding(text,voc_size=vs,split=sp,tokenizer=tok,training=tr)
    
    if choose_model=='SMHA':
        model=fats.SMHA_Linear_classifier(torch.Size([512,768]),8,768)
        model.load_state_dict(torch.load('SMHA_Linear_classifier.pt',map_location=torch.device('cpu')))
        model.eval()
    
    elif choose_model=='Simple':
        model=fats.Simple_Classifier(camem.config.hidden_size)
        model.load_state_dict(torch.load('Simple_Classifier.pt',map_location=torch.device('cpu')))
        model.eval()
    
    elif choose_model=='Net':
        model=fats.Net(2**8,2**6,2,2,2,2)
        model.load_state_dict(torch.load('Net.pt',map_location=torch.device('cpu')))
        model.eval()    
        
    elif choose_model=='Multi':
        model=fats.Multi_Linear_Classifier(camem.config.hidden_size)
        model.load_state_dict(torch.load('Multi_Linear_Classifier.pt',map_location=torch.device('cpu')))
        model.eval()    
    
    y=model(camem(torch.Tensor(dico['input']).long(),torch.Tensor(dico['mask']).long()).last_hidden_state)
    t=torch.mul(dico['mask_cls'],y).topk(3)
    vec=[(dico['mask_cls'][i]==torch.tensor(1)).nonzero() for i in range(len(dico['mask_cls']))]
    values=t[0]
    indice=t[1]
    Ve=[]
    Va=[]
    I=[]
    ind=0
    for i in range(len(dico['trace'])):
        Ve.append(vec[ind:ind+dico['trace'][i]])
        Va.append(values[ind:ind+dico['trace'][i]])
        I.append(indice[ind:ind+dico['trace'][i]])
        ind+=dico['trace'][i]
        
    Va=[v.reshape(-1) for v in Va]
    I=[v.reshape(-1) for v in I]
    
    I2=[torch.cat([I[k][i*3:(i+1)*3]+512*i for i in range(int(len(I[k])/3))]) for k in range(len(I))]
    Ve2=[torch.cat([Ve[k][i]+512*i for i in range(len(Ve[k]))]).squeeze(1) for k in range(len(Ve))]

    index=[torch.div(Va[i],Va[i].sum()).topk(k=3)[1] for i in range(len(Va))]
    vrai_index=[I2[i][index[i]] for i in range(len(texte))]

    rindex=[torch.cat([(Ve2[h]==vrai_index[h][i]).nonzero().squeeze(1) for i in range(k)]) for h in range(len(Ve2))]
    
    a=[i for i in range(len(texte[0]))]
    for i in t1ouf:
        a.remove(i)
    
    
    if get_score_only:
        return rindex,vrai_index,Ve2,I2
    else:
        resu=[[text[i][k] for k in rindex[i]] for i in range(len(text))]
        return resu
        

In [5]:
r=fats.make_DL_resume(texte,cpu=2,choose_model='Simple',get_score_only=True)
r

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


UnboundLocalError: local variable 'train_input_ids' referenced before assignment

In [8]:
for k in range(len(text)):
    for i in r[k]:
        try:
            text[k][i]
        except:
            print(k,i)
            continue

7 tensor(18)


In [4]:
ME=fats.Make_Extractive(cpu=5).make_encoding(text=text,tokenizer=tok,training=False)
(torch.Tensor(ME['input'])==torch.tensor(5)).int()

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        ...,
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)